In [31]:
# Data prep
import os, shutil
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from random import random
import pandas as pd
import re
import tensorflow_addons as tfa

# data loading config
batch_size = 64
img_height = 128
img_width = 128
dataPath = 'oxford_flower_102'
labels = 'inferred'
label_mode = 'categorical'  # one hot encoding
color_mode = 'rgb'
shuffle = True
seed = 69
test_split = 0.2  # split into train and test (NOT val), 0-1
AUTOTUNE = tf.data.AUTOTUNE

train_df = pd.read_csv('image_to_label.csv', names=['picture', 'label'])
with open('102_flower_labels.txt') as f:
    regex = re.compile('[^a-zA-Z\s-]')
    classes=[]
    for line in f:
        classes += [regex.sub('', line.replace('\'', '').strip())]


def split_data(dataPath):
    for root, dirs, files in os.walk(dataPath):
        for name in files:
            randomNum = random()
            row = train_df.iloc[train_df.index[train_df['picture'] == name]]
            if randomNum <= test_split:
                os.makedirs('test\\'+classes[row.label.tolist()[0]-1]+'\\', exist_ok=True)
                shutil.move(root+'\\'+name, 'test\\'+classes[row.label.tolist()[0]-1]+'\\')
            elif test_split< randomNum <= test_split + test_split * (1-test_split):
                os.makedirs('val\\'+classes[row.label.tolist()[0]-1]+'\\', exist_ok=True)
                shutil.move(root+'\\'+name, 'val\\'+classes[row.label.tolist()[0]-1]+'\\')
            else:
                os.makedirs('train\\'+classes[row.label.tolist()[0]-1]+'\\', exist_ok=True)
                shutil.move(root+'\\'+name, 'train\\'+classes[row.label.tolist()[0]-1]+'\\')

# split_data(dataPath)  # Only need to run this once

train_datagen = ImageDataGenerator( # Do 0-1 scaling as a layer so that saved model includes it
    rotation_range=20, width_shift_range=10.0,
    height_shift_range=10.0, brightness_range=(-0.2, 0.2), shear_range=0.2, zoom_range=0.2,
    channel_shift_range=0.2, fill_mode='nearest', horizontal_flip=True, vertical_flip=True)


print('Training data:')
train_generator = train_datagen.flow_from_directory('train', target_size=(img_height, img_width), batch_size=batch_size,
                                                    color_mode=color_mode, class_mode=label_mode, shuffle=shuffle, seed=seed)
print('\nValidation data:')
val = keras.preprocessing.image_dataset_from_directory('val', labels=labels, label_mode=label_mode,
color_mode=color_mode, shuffle=shuffle, seed=seed, image_size=(img_height, img_width), batch_size=batch_size)
val_class_names = val.class_names
print('\nTesting data:')
test = keras.preprocessing.image_dataset_from_directory('test', labels=labels, label_mode=label_mode,
color_mode=color_mode, shuffle=shuffle, seed=seed, image_size=(img_height, img_width), batch_size=batch_size)
test_class_names = test.class_names

val = val.cache().prefetch(buffer_size=AUTOTUNE)
test = test.cache().prefetch(buffer_size=AUTOTUNE)

assert list(train_generator.class_indices.keys()) == val_class_names == test_class_names, 'Classes mismatch!'
classes = list(train_generator.class_indices.keys())
print('\nClasses:', classes)

Training data:
Found 5248 images belonging to 102 classes.

Validation data:
Found 1320 files belonging to 102 classes.

Testing data:
Found 1621 files belonging to 102 classes.

Classes: ['alpine sea holly', 'anthurium', 'artichoke', 'azalea', 'ball moss', 'balloon flower', 'barbeton daisy', 'bearded iris', 'bee balm', 'bird of paradise', 'bishop of llandaff', 'black-eyed susan', 'blackberry lily', 'blanket flower', 'bolero deep blue', 'bougainvillea', 'bromelia', 'buttercup', 'californian poppy', 'camellia', 'canna lily', 'canterbury bells', 'cape flower', 'carnation', 'cautleya spicata', 'clematis', 'colts foot', 'columbine', 'common dandelion', 'corn poppy', 'cyclamen', 'daffodil', 'desert-rose', 'english marigold', 'fire lily', 'foxglove', 'frangipani', 'fritillary', 'garden phlox', 'gaura', 'gazania', 'geranium', 'giant white arum lily', 'globe thistle', 'globe-flower', 'grape hyacinth', 'great masterwort', 'hard-leaved pocket orchid', 'hibiscus', 'hippeastrum', 'japanese anemone

In [17]:
class YiqinIsGay(layers.Layer):
    def __init__(self, filters, kernel_size):
        self.filters = filters
        self.kernel_size = kernel_size
        super(YiqinIsGay, self).__init__()
    def build(self, input_shape):
        shape = list(self.kernel_size) + [input_shape[-1], self.filters]
        print(shape)
        self.kernel = self.add_weight(name='YiqinIsHeavy', shape=shape, initializer='glorot_uniform', trainable=True)
        super(YiqinIsGay, self).build(input_shape)
    def call(self, x):
        # print('Yiqin is Gay')
        self.kernel = self.kernel * tf.math.sqrt(2/(self.kernel_size[0] * self.kernel_size[1] * self.filters))
        return tf.nn.conv2d(x, self.kernel, strides=1, padding='SAME')
    def compute_output_shape(self, input_shape):
        return input_shape[:-1] + (self.filters,)

In [32]:
opt = keras.optimizers.Adam(lr=1e-3, epsilon=1e-6, clipvalue=0.8)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
epoch = 100
batch_size = 32
metrics = ['accuracy'] # , tfa.metrics.F1Score(num_classes=len(classes))
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, patience=15, verbose=1,
                                     mode='auto', baseline=None, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint('./best_model',monitor='val_accuracy',save_best_only=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.05, patience=10, verbose=1)
]

# Model
xInput = layers.Input((img_height, img_width, 3))
x = layers.experimental.preprocessing.Rescaling(1./255)(xInput)  # fit 0-255 into 0 and 1
x = layers.Conv2D(32, (3, 3))(x)
x = layers.Activation('relu')(x)
x = layers.MaxPooling2D(pool_size=(2,2))(x)
x = layers.Conv2D(64, (3, 3))(x)
x = layers.Activation('relu')(x)
x = layers.MaxPooling2D(pool_size=(2,2))(x)
x = layers.Conv2D(128, (3, 3))(x)
x = layers.Activation('relu')(x)
x = layers.MaxPooling2D(pool_size=(2,2))(x)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Activation('relu')(x)
x = layers.Dropout(0.4)(x)
xOutput = layers.Dense(len(classes), activation='softmax')(x)
model = keras.Model(xInput, xOutput)
model.compile(optimizer=opt, loss=loss, metrics=metrics)
model.summary()
model.fit(train_generator, batch_size=batch_size, steps_per_epoch=train_generator.samples, epochs=epoch, validation_data=val, callbacks=callbacks, verbose=1)

Model: "model_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_22 (InputLayer)        [(None, 128, 128, 3)]     0         
_________________________________________________________________
rescaling_20 (Rescaling)     (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_55 (Conv2D)           (None, 126, 126, 32)      896       
_________________________________________________________________
activation_85 (Activation)   (None, 126, 126, 32)      0         
_________________________________________________________________
max_pooling2d_58 (MaxPooling (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_56 (Conv2D)           (None, 61, 61, 64)        18496     
_________________________________________________________________
activation_86 (Activation)   (None, 61, 61, 64)        0  

In [33]:
model.evaluate(test)
model.save('oxford_flower_102')

26/26 [==============================] - 2s 53ms/step - loss: 4.4682 - accuracy: 0.0537